In [1]:
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [8]:
def createModelMatrix(matrixDim,sparsity):
    #matrixDim=10
    #sparsity = 0.01
    Amat = np.zeros((matrixDim,matrixDim))
    for i in range(0,matrixDim):
        Amat[i,i] = i + 1
    Amat = Amat + sparsity*np.random.randint(20,size=(matrixDim,matrixDim)) 
    #print("Amat: \n",Amat)
    u,v=LA.eig(Amat)
    print('Correct roots: ',np.sort(u))
    return Amat

def defineAllVariables(Amat,matrixDim): #defines single vector p,q,eps,H0,V from a given matrix and its dimension
    p=np.zeros((matrixDim,1))
    q=np.zeros((matrixDim,matrixDim-1))
    print(np.shape(p),np.shape(q))
    diagMat=np.diag(Amat)
    #print('diagonal of Matrix: ',diagMat)
    
    newDiag=[]
    for i in range(matrixDim):
        for j in range(matrixDim):
            if i==j:
                newDiag.append(Amat[i,j])
                
    #print('new diagonal of matrix: ',newDiag)
    
    lowestElements=np.argsort(diagMat)
    #print('lowest element index:',lowestElements)
    #print('actual lowest element of Hbar:',diagMat[lowestElements[0]])

    p[lowestElements[0],0]=1.0
    #print('p',p)
    counter=0
    for i in range(len(lowestElements)):
        if i==lowestElements[0]:
            continue
        else:
            q[lowestElements[i],counter]=1.0
            counter=counter+1


        
        
    H0=np.diag(np.diag(Amat))#np.matmul(np.matmul(projP,Amat),projP)
    V=Amat-H0
    #print('V \n',V)
    #print('original Guess:',diagMat[0])
    guessEps=diagMat[lowestElements[0]] -0.01
    eps=guessEps
    print('init Guess:',eps)
    
    return p,q,H0,V,eps

def createT0_projQ(q,qDim,H0Dim,eps,H0): #qDim>0
    #print('q',q,eps,qDim)
    epsMat=np.zeros((H0Dim,H0Dim))
    denom=np.zeros((H0Dim,H0Dim))
    np.fill_diagonal(epsMat,eps)
    
    sumMatrix=np.zeros((H0Dim,H0Dim))
    for i in range(qDim):
        Q=np.outer(q[:,i],q[:,i])
        #print('Q outer:',Q)
        QH0Q=np.matmul(np.matmul(Q,H0),Q)
        denom=epsMat-QH0Q
        for x in range(H0Dim):
            if (abs(denom[x,x]) < 0.0001):
                if (denom[x,x]< 0.00000000000000001):
                    factor=-1.0
                    denom[x,x]=factor*0.0001
                else:
                    factor=1.0
                    denom[x,x]=factor*0.0001
            denom[x,x]=1.0/denom[x,x]
                
        #denom=inv(denom)
        sumMatrix=sumMatrix+np.matmul(np.matmul(Q,denom),Q) #q[:,i],denom),q[:,i].T)
    #print('T0 value is: ',sumMatrix)
    return sumMatrix


def extendPSpace(T0,V,p,order):
    T0V=np.matmul(T0,V)
    tmp=T0V
    
    for i in range(1,order+1):
        print('extending guess for order: ',i)
        tmp=np.matmul(tmp,T0V)
        
    correctionVec=np.matmul(tmp,p)
    #print('Correction vector: \n',correctionVec)
    return correctionVec

def computeRHR(fullSpace,H,HDim,order):
    overlap=fullSpace[:,:order+1].T @ fullSpace[:,:order+1]
    #print('overlap',overlap)
    s,u=LA.eig(overlap)
    #print('little s:',s)
    sDiag=np.zeros((order+1,order+1))#
    np.fill_diagonal(sDiag,s)
    for i in range(order+1):
        sDiag[i,i]=1.0/np.sqrt(sDiag[i,i])
    #print('sdiag:',sDiag)
        
    X=u.T @ sDiag
    X=X @ u
    RHR=(fullSpace[:,:order+1].T @ H) @ fullSpace[:,:order+1]
    Hprime=X.T @ RHR
    Hprime=Hprime @ X
    try:
        roots,vecs=LA.eig(Hprime)
        indx=roots.argsort()
        theta=roots[indx]
        vecs=vecs[:,indx] # these are C' ; must convert back to C, ie XC'=C
        transformedVecs=X@vecs # this is the back transform step
        usableRoots=[ elem for elem in theta if elem > 0.15 ]
        usableIndx=[x for x in range(len(theta)) if theta[x] > 0.15]
        #print('Usable roots from <R|H|R>: ',usableRoots)
        #print('Usable vecs from <R|H|R>: ',transformedVecs[:,usableIndx])
    except np.linalg.LinAlgError as err:
        print('inf or NaN inside the code performing delta^(-1/2)Hdelta^(-1/2) diagonalization!!')
        print("Formal error message: ")
        print(err)
    
    
    ## test orthogonalizing fullspace
    qq,r=LA.qr(fullSpace[:,:order+1])
    #print('orthogonalized contribuition to |R>:',qq[:,order])
    RHR=(qq.T @ H) @ qq
    print('RHR: \n',RHR)
    roots,vecs=LA.eig(RHR)
    print('Roots from <R|H|R> via QR:',np.sort(roots))
    indx=roots.argsort()
    theta=roots[indx]
    vecs=vecs[:,indx]
    #print('C_i vecs: ',vecs)
    usableRoots=[ elem for elem in theta if elem > 0.15 ]
    usableIndx=[x for x in range(len(theta)) if theta[x] > 0.15]
    #print('C_i vecs: ',vecs[:,usableIndx])
    return usableRoots,vecs[:,usableIndx]#usableRoots,transformedVecs[:,usableIndx]   
        
        
def runVariationalAlgo(H,Hdim):
    p,q,H0,V,eps=defineAllVariables(H,Hdim)
    #print('H0:',H0)
    fullSpace=np.zeros((Hdim,Hdim))
    #print('shape of FS and p',np.shape(fullSpace[:,0]),np.shape(p))
    fullSpace[:,[0]]=p
    order=0
    qDim=Hdim-1
    Hbkup=H
    #print('initial guess H0: ',eps)
    finalEps=[]
    tmpRoots=np.zeros((Hdim,Hdim))
    for i in range(10):
        print('*********************************')
        print('*********************************')
        print('***** Iteration: ',i)
        T0=createT0_projQ(q,qDim,Hdim,eps,H0)
        pertVec=extendPSpace(T0,V,p,order) # first order correction
        order=order+1
        normPertVec=LA.norm(pertVec)
        pertVec=(1.000000000000/normPertVec)*pertVec
        fullSpace[:,[order]]=pertVec
        
        ## try orthogonalizing fullSpace
        qq,r=LA.qr(fullSpace[:,:order+1])
        fullSpace[:,:order+1]=qq
        ##
        ritzRoots,ritzVecs=computeRHR(fullSpace,H,Hdim,order)
        #print('ritzVecs:',ritzVecs)
        print('ritzRoots:',np.sort(ritzRoots))
        transformedVec=fullSpace[:,:order+1]@ritzVecs[:,0]
        print('<T|H|T>= ?',(transformedVec.T @ H) @transformedVec)
        np.fill_diagonal(tmpRoots,np.sort(ritzRoots)[0])
        residualVec=(H-tmpRoots)@transformedVec
        normResid=LA.norm(residualVec)
        print('Norm of residual: ',normResid)
        eps=np.sort(ritzRoots)[0]
        finalEps.append(eps)
        
    return finalEps
       


In [9]:
#HeH+ AUG-CC-PVTZ
cisHeH=np.loadtxt('HeHaugccpvtz.txt')
print(cisHeH)
finalCisHeH=np.zeros((45,45)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(45):
    for j in range(45):
        #print('i,j:',i,j)
        finalCisHeH[i,j]=cisHeH[counter]
        #finalCisHeH[j,i]=finalCisHeH[i,j]
        counter=counter+1
        
print('finalCISheH',finalCisHeH)
guessRoots,guessVecs=LA.eig(finalCisHeH)
print('HeH+ CIS roots:',np.sort(guessRoots))
print('HeH+ correct lowest root: ',np.sort(guessRoots)[0])

finalEps=runVariationalAlgo(finalCisHeH,45)

[0.82775019 0.0371169  0.         ... 0.         0.02724339 9.57486946]
finalCISheH [[ 8.27750187e-01  3.71169013e-02  0.00000000e+00 ...  0.00000000e+00
   1.50872147e-02 -1.65567836e-03]
 [ 3.71169013e-02  1.15491469e+00  0.00000000e+00 ...  0.00000000e+00
  -5.43788560e-03  6.58663239e-03]
 [ 0.00000000e+00  0.00000000e+00  1.11166601e+00 ...  1.09100477e-02
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  1.09100477e-02 ...  8.51138385e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.50872147e-02 -5.43788560e-03  0.00000000e+00 ...  0.00000000e+00
   8.70483162e+00  2.72433924e-02]
 [-1.65567836e-03  6.58663239e-03  0.00000000e+00 ...  0.00000000e+00
   2.72433924e-02  9.57486946e+00]]
HeH+ CIS roots: [0.81675677 1.08916656 1.08916656 1.12228085 1.25085294 1.32343575
 1.37571255 1.37571255 1.37951985 1.50846121 1.6047357  1.6047357
 1.69199282 1.69199282 1.80951649 1.86518314 1.86518314 1.92895166
 2.1206591  2.31697298 2.31697298 2.53440247 2.53440247 2.623012

In [ ]:
print('HeH aug-cc-pvtz basis: ',finalEps)
0.023142729774758113

In [ ]:
#HeH+
cisHeH=np.loadtxt('cisHeH.txt')
print(cisHeH)
finalCisHeH=np.zeros((11,11)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(11):
    for j in range(i,11):
        print('i,j:',i,j)
        finalCisHeH[i,j]=cisHeH[counter]
        finalCisHeH[j,i]=finalCisHeH[i,j]
        counter=counter+1
        
print('finalCISheH',finalCisHeH)
guessRoots,guessVecs=LA.eig(finalCisHeH)
print('HeH+ CIS roots:',np.sort(guessRoots))

tmp=runVariationalAlgo(finalCisHeH,11)

In [ ]:
print('CIS iterations for HeH+:',tmp)

In [ ]:
# Water CIS
# Water CIS
water=np.loadtxt('waterCIS.txt')
print(water)
finalwater=np.zeros((100,100)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(100):
    for j in range(100):
        #print('i,j:',i,j)
        finalwater[i,j]=water[counter]
        counter=counter+1
        
print('water',finalwater)
guessRoots,guessVecs=LA.eig(finalwater)
print('water CIS roots:',np.sort(guessRoots))
eps=runVariationalAlgo(finalwater,100)
print('Convergence Info: ',eps)
0.37098005


In [ ]:
#water dimer
# Water DIMER CIS
water=np.loadtxt('waterDimerCIS.txt')
print(water)
finalwater=np.zeros((400,400)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(400):
    for j in range(400):
        #print('i,j:',i,j)
        finalwater[i,j]=water[counter]
        counter=counter+1
        
print('water',finalwater)
guessRoots,guessVecs=LA.eig(finalwater)
print('water CIS roots:',np.sort(guessRoots))
eps=runVariationalAlgo(finalwater,400)

In [ ]:
print(eps)
x=np.arange(1,len(eps)+1)
print(x)
plt.plot(x,eps,'o')
plt.show()

In [ ]:
error=[]
for i in range(len(eps)):
    tmp=abs(0.38009787-eps[i])
    error.append(tmp)
    
print(error)
print(np.log10(error))
plt.plot(x,np.log10(error),'o')
plt.xlabel('Iteration')
plt.ylabel('Log10 Error')
plt.show()

In [ ]:
0.38009787-0.3800982243192757

In [ ]:
0.38009787-0.380098223875706

In [ ]:
0.38009822387570497-0.38009787

In [ ]:
0.38009787-0.380099167857653

In [ ]:
print(float(1.0))

In [ ]:
print(repr(1.000))

In [ ]:
from decimal import Decimal 
print(Decimal(1.0))

In [3]:
#water dimer
# Water DIMER CIS
water=np.loadtxt('waterDimerAugccpvtz.txt')
print(water)
finalwater=np.zeros((1740,1740)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(1740):
    for j in range(1740):
        #print('i,j:',i,j)
        finalwater[i,j]=water[counter]
        counter=counter+1
        
print('water',finalwater)
guessRoots,guessVecs=LA.eig(finalwater)
print('water CIS roots:',np.sort(guessRoots))
eps=runVariationalAlgo(finalwater,1740)

[ 2.04448155e+01  3.79777865e-04 -1.67875521e-02 ... -3.63677849e-03
 -2.40661852e-01  1.69260432e+01]
water [[ 2.04448155e+01  3.79777865e-04 -1.67875521e-02 ... -7.66179680e-07
   4.51121190e-08 -4.54503646e-08]
 [ 3.79777865e-04  2.04989511e+01 -7.04398924e-04 ... -2.76024691e-06
   1.13849363e-05 -6.96382652e-06]
 [-1.67875521e-02 -7.04398924e-04  2.05319301e+01 ... -3.91912076e-07
   2.44470410e-07 -1.55317560e-07]
 ...
 [-7.66179680e-07 -2.76024691e-06 -3.91912076e-07 ...  8.74147676e+00
   1.14173178e-02 -3.63677849e-03]
 [ 4.51121190e-08  1.13849363e-05  2.44470410e-07 ...  1.14173178e-02
   1.60616780e+01 -2.40661852e-01]
 [-4.54503646e-08 -6.96382652e-06 -1.55317560e-07 ... -3.63677849e-03
  -2.40661852e-01  1.69260432e+01]]
water CIS roots: [ 0.32209284  0.33606141  0.3809051  ... 35.6503957  36.83515592
 37.22444782]
(1740, 1) (1740, 1739)
init Guess: 0.376090425577361
*********************************
*********************************
***** Iteration:  0
RHR: 
 [[ 0.386090

extending guess for order:  1
extending guess for order:  2
extending guess for order:  3
extending guess for order:  4
extending guess for order:  5
extending guess for order:  6
extending guess for order:  7
extending guess for order:  8
extending guess for order:  9
RHR: 
 [[ 3.86090426e-01  3.80553213e-02 -2.20701653e-02 -3.45406142e-02
  -1.30708006e-03  1.37451521e-02  1.11713208e-03 -3.54083922e-03
  -3.10689892e-03  3.86480117e-03  7.31043061e-03]
 [ 3.80553213e-02  3.83807358e-01  4.58236503e-02 -1.73083888e-02
  -1.96647628e-02  4.81133973e-03 -5.86160701e-03  8.88175007e-03
   1.42124579e-02  4.85722168e-04 -4.32641045e-03]
 [-2.20701653e-02  4.58236503e-02  4.47047692e-01 -4.02974213e-02
  -2.20382218e-02 -5.23475722e-02 -8.41459534e-03  2.92382759e-02
   1.47671630e-02 -2.38072501e-02 -4.78988362e-03]
 [-3.45406142e-02 -1.73083888e-02 -4.02974213e-02  4.65487664e-01
   2.63323954e-02  6.69413523e-02 -1.47612715e-02 -2.05551051e-02
   1.36872157e-02  2.85771764e-02 -3.06089

In [ ]:
0.

In [ ]:
0.32209284